# Adaptive RAG Router - Training DemoThis notebook demonstrates training the **Adaptive RAG Router** on CLINC150 dataset.Works on both GitHub and Kaggle!

In [ ]:
import sysimport os# Install package if running in Kaggle/Colabif 'kaggle' in os.environ.get('KAGGLE_URL_BASE', '') or 'COLAB_GPU' in os.environ:    !pip install -q transformers datasets peft accelerate torch# For local development, the package would be installed via setup.py

In [ ]:
import torchimport pandas as pdimport matplotlib.pyplot as pltfrom adaptive_rag_router import (    CLINC150DataLoader,    AdaptiveRAGRouter,    create_router_model,    ModelTrainer)print("🚀 Adaptive RAG Router - Training Demo")print(f"PyTorch: {torch.__version__}")print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
print("🎯 Quick Demo - Model Prediction")# Initialize a pretrained modelmodel = create_router_model("distilbert")# Test predictionstest_queries = [    "What's my account balance?",    "I need to transfer money between accounts",    "What's the weather like today?",    "Can you help me with my credit card application?",]results = model.predict(test_queries)print("\n📊 Prediction Results:")for i, query in enumerate(test_queries):    domain = results["domains"][i]    confidence = results["confidences"][i]    print(f" '{query}' → {domain} ({confidence:.3f})")

In [ ]:
print("\n📈 Data Exploration")data_loader = CLINC150DataLoader()train_loader, val_loader, test_loader = data_loader.get_data_loaders(batch_size=8)print(f"Training batches: {len(train_loader)}")print(f"Validation batches: {len(val_loader)}")print(f"Test batches: {len(test_loader)}")# Show domain distributiondataset = data_loader.load_dataset("train", sample_size=1000)domains = [data_loader.extract_domain_from_intent(item['intent']) for item in dataset]from collections import Counterdomain_counts = Counter(domains)plt.figure(figsize=(10, 6))plt.bar(domain_counts.keys(), domain_counts.values())plt.title('Domain Distribution in CLINC150')plt.xticks(rotation=45)plt.tight_layout()plt.show()

In [ ]:
print("\n🎓 Model Training")trainer = ModelTrainer()# For quick demo in cloud environments, use smaller samplesif 'KAGGLE_KERNEL_RUN_TYPE' in os.environ or 'COLAB_GPU' in os.environ:    print("🏃 Running quick demo training (cloud-optimized)...")    results = trainer.train_quick_demo()else:    print("🔧 Running full training...")    results = trainer.train_model("distilbert")print(f"✅ Training completed! Accuracy: {results['test_accuracy']:.4f}")

In [ ]:
print("\n📊 Model Evaluation")# Load the trained modeltrained_model = AdaptiveRAGRouter()trained_model.load(results['output_dir'])# Test on sample queriestest_queries = [    "What's my current balance?",    "I want to pay my credit card bill",    "What's the weather forecast?",    "Book a flight to London",    "Reset my password please"]predictions = trained_model.predict(test_queries)print("Model Predictions on Test Queries:")for i, query in enumerate(test_queries):    print(f" {query:<40} → {predictions['domains'][i]:<20} (conf: {predictions['confidences'][i]:.3f})")print("\n🎉 Demo completed successfully!")